Action3	"使用Gensim中的Word2Vec对三国演义进行Word Embedding，分析和曹操最相近的词有哪些，曹操+刘备-张飞=?
数据集：three_kingdoms.txt"			
1、完成代码，结果正确（20points）	

In [4]:
# -*-coding: utf-8 -*-
# 对txt文件进行中文分词
import jieba
import os
from utils import files_processing

# 源文件所在目录
source_folder = './three_kingdoms/source'
segment_folder = './three_kingdoms/segment'

# 字词分割，对整个文件内容进行字词分割
def segment_lines(file_list,segment_out_dir,stopwords=[]):
    for i,file in enumerate(file_list):
        segment_out_name=os.path.join(segment_out_dir,'segment_{}.txt'.format(i))
        with open(file, 'rb') as f:
            document = f.read()
            document_cut = jieba.cut(document)
            sentence_segment=[]
            for word in document_cut:
                if word not in stopwords:
                    sentence_segment.append(word)
            result = ' '.join(sentence_segment)
            result = result.encode('utf-8')
            with open(segment_out_name, 'wb') as f2:
                f2.write(result)

# 对source中的txt文件进行分词，输出到segment目录中
file_list=files_processing.get_files_list(source_folder, postfix='*.txt')
segment_lines(file_list, segment_folder)

In [10]:
# -*-coding: utf-8 -*-
# 先运行 word_seg进行中文分词，然后再进行word_similarity计算
# 将Word转换成Vec，然后计算相似度 
from gensim.models import word2vec
import multiprocessing

# 如果目录中有多个文件，可以使用PathLineSentences
segment_folder = './three_kingdoms/segment'
sentences = word2vec.PathLineSentences(segment_folder)

# 设置模型参数，进行训练
model = word2vec.Word2Vec(sentences, size=100, window=3, min_count=1)
print(model.wv.similarity('曹操', '刘备'))
print(model.wv.similarity('曹操', '曹孟德'))
print(model.wv.most_similar(positive=['曹操', '刘备'], negative=['张飞']))

0.9791641
0.93517995
[('丞相', 0.9973059296607971), ('某', 0.9961576461791992), ('前者', 0.9956419467926025), ('蒙', 0.9945294857025146), ('奏', 0.9938101172447205), ('主公', 0.9937384724617004), ('瑾', 0.9935469031333923), ('此言', 0.9935237765312195), ('臣', 0.9933129549026489), ('问', 0.9931826591491699)]


In [11]:
# 设置模型参数，进行训练
model2 = word2vec.Word2Vec(sentences, size=128, window=5, min_count=5, workers=multiprocessing.cpu_count())
# 保存模型
model2.save('./models/word2Vec.model')
print(model.wv.similarity('曹操', '刘备'))
print(model.wv.similarity('曹操', '曹孟德'))
print(model.wv.most_similar(positive=['曹操', '刘备'], negative=['张飞']))

0.9791641
0.93517995
[('丞相', 0.9973059296607971), ('某', 0.9961576461791992), ('前者', 0.9956419467926025), ('蒙', 0.9945294857025146), ('奏', 0.9938101172447205), ('主公', 0.9937384724617004), ('瑾', 0.9935469031333923), ('此言', 0.9935237765312195), ('臣', 0.9933129549026489), ('问', 0.9931826591491699)]
